In [1]:
from kimport import *
import kplot.kg
import corona as cr
import gootools as gt

irdap_main_path = "D:/Science/LBV/test_irdap" 

In [2]:
def mean_angle(angles):
    #assume starting in degrees
    a = np.deg2rad(angles)
    #compute x,y components
    y = np.mean(np.sin(a),axis=0)
    x = np.mean(np.cos(a),axis=0)
    #go back to angles
    mean_angle = np.rad2deg(np.arctan2(y,x))
    return mean_angle    

---
### Stacking Observations & Derotating
---

In [3]:
# initialize array to store final images
images = list()
errors = list()
# cycle through each object
for path in glob(irdap_main_path+'/*'): 
    print('\nStacking images of {}'.format(path.split('\\')[-1]))
    # grab preprocessed left and right files
    fleft = glob(path+'/preprocessed/*left*.fits')[0]
    fright = glob(path+'/preprocessed/*right*.fits')[0]
    cubes = [cr.data(fleft),cr.data(fright)]
    # cycle through each side
    for j,cube in enumerate(cubes):
        # grab the raw files, since those have the rotation angles
        raw_files = np.array([f    for f in glob(path+'/raw/*OBJECT*.fits') if 'CENTER' not in f])
        # initialize array so np._r works below
        derotated_image = [np.zeros((1024,1024))]
        # cycle through each frame, matching it with the observation which has its rotation angle
        for i,(image,raw) in enumerate(zip(cube,raw_files)):
            gt.progress_bar(j*len(raw_files)+i+1,2*len(raw_files))
            header = cr.header(raw)
            angle = mean_angle(np.array([float(header['HIERARCH ESO TEL PARANG START']),float(header['HIERARCH ESO TEL PARANG END'])])) + cr.true_north_correction + cr.pupil_offset
            new_image = rotate(image,-1*angle,reshape=False)
            derotated_image = np.r_[derotated_image,[new_image]]
    d = np.mean(derotated_image[1:],axis=0)
    e = np.std(derotated_image[1:],axis=0)
    images.append(d)
    errors.append(e)


Stacking images of AS-314
|█████████████████████████████████████████████████████████████████████████████████████████████████   |97.1%
Stacking images of HD-160529
|██████████████████████████████████████████████████████████████████████████████████████████████████  |99.0%
Stacking images of HD-168607
|██████████████████████████████████████████████████████████████████████████████████████████████████  |98.5%
Stacking images of HD-168625
|█████████████████████████████████████████████████████████████████████████████████████████████████   |97.1%
Stacking images of HD-316285
|█████████████████████████████████████████████████████████████████████████████████████████████████   |97.1%
Stacking images of HD-326823
|█████████████████████████████████████████████████████████████████████████████████████████████████   |97.1%
Stacking images of MWC-314
|█████████████████████████████████████████████████████████████████████████████████████████████████   |97.1%
Stacking images of ZETA-SCO
|████████████████

Save to file

In [4]:
for image,error,name in zip(images,errors,cr.targets):
    np.savetxt('data/{}.txt'.format(name),image,delimiter=',')
    np.savetxt('data/{}_error.txt'.format(name),error,delimiter=',')

---